# Introduction Of One-Multi Solution

**This strategy is based on the idea that rather than providing passengers with a direct flight to their destination, we will attempt to shorten their wait time by providing them with the option of connecting flights, which will shorten the total time. However, we are only allowing one connecting flight because providing more will lower passenger satisfaction.**

**Mount the drive here**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import datetime

### **Data Loading and Preprocessing Block**

In [3]:
# Data Loading

df_pnr_book = pd.read_csv("/content/drive/MyDrive/TEAM-69/Final Dataset/PNRB-ZZ-20231208_062017.csv")
df_pnr_pass = pd.read_csv("/content/drive/MyDrive/TEAM-69/Final Dataset/PNRP-ZZ-20231208_111136.csv")
df_schedule = pd.read_csv('/content/drive/MyDrive/TEAM-69/Final Dataset/SCH-ZZ-20231208_035117.csv')
df_inventory = pd.read_csv('/content/drive/MyDrive/TEAM-69/Final Dataset/INV-ZZ-20231208_041852.csv')

# Data Preprocessing

date_columns = ['DEP_DTML', 'ARR_DTML', 'DEP_DTMZ', 'ARR_DTMZ']
for column in date_columns:
    df_pnr_book[column] = pd.to_datetime(df_pnr_book[column], errors='coerce').dt.strftime('%m/%d/%Y %H:%M')
df_pnr_book['DEP_DT'] = pd.to_datetime(df_pnr_book['DEP_DT'], errors='coerce').dt.strftime('%m/%d/%Y')
#For Inventory
df_inventory['DepartureDate'] = pd.to_datetime(df_inventory['DepartureDate'], errors='coerce').dt.strftime('%m/%d/%Y')
date_columns = ['DepartureDateTime', 'ArrivalDateTime']
for column in date_columns:
    df_inventory[column] = pd.to_datetime(df_inventory[column], errors='coerce').dt.strftime('%Y/%m/%d %H:%M:%S')
date_columns = ['DepartureDateTime', 'ArrivalDateTime']
df_inventory[date_columns] = df_inventory[date_columns].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')

### **Function Block for obtaining all the impacted PNRS resulting from a specific flight cancellation in the priority order determined by their score**


In [4]:
def get_ITN_DEST(df):
    df[['SEG_SEQ', 'SEG_TOTAL']] = df[['SEG_SEQ', 'SEG_TOTAL']].apply(pd.to_numeric, errors='coerce')
    max_dest_per_recloc = df.loc[df['SEG_SEQ'] == df['SEG_TOTAL']].groupby('RECLOC')['DEST_CD'].first().reset_index()
    df = pd.merge(df, max_dest_per_recloc, on='RECLOC', how='left', suffixes=('', '_ITN_DEST'))
    return df

def process_pnr_data(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date):

    dept_time = df_schedule[(df_schedule['FlightNumber'] == flight_number) & (df_schedule['AircraftTailNumber'] == tail_number)]['DepartureTime'].iloc[0]
    date_time = date + ' ' + dept_time
    print(date_time)

    def cal_affected_persons(flt_number, date_time):
        dict={}
        affect_persons = pd.DataFrame(columns=df_pnr_book.columns)

        for i in range(len(df_pnr_book)):
            if df_pnr_book.loc[i, "RECLOC"] in dict.keys():
                dict[df_pnr_book.loc[i, "RECLOC"]] = max(dict[df_pnr_book.loc[i, "RECLOC"]], df_pnr_book.loc[i, "SEG_SEQ"])
            else:
                dict[df_pnr_book.loc[i, "RECLOC"]] = df_pnr_book.loc[i, "SEG_SEQ"]

            if df_pnr_book.loc[i, "FLT_NUM"] == flt_number and df_pnr_book.loc[i, "DEP_DTML"] == date_time:
                affect_persons = affect_persons.append(df_pnr_book.loc[i])


        li = []
        for i in range(len(affect_persons)):
            if(dict[affect_persons.iloc[i]["RECLOC"]]):
                li.append(dict[affect_persons.iloc[i]["RECLOC"]])
        affect_persons["MAX_SEG"] = li
        return affect_persons

    def calc_ssr(flight_number, date_time):
        affect_person = cal_affected_persons(flight_number, date_time)
        li = []
        for i in range(len(affect_person)):
            li.append(0)
        affect_person["count_ssr"] = li
        for i in range(len(affect_person)):
            for j in range(len(df_pnr_pass)):
                if df_pnr_pass.loc[j, "RECLOC"] == affect_person.iloc[i]["RECLOC"]:
                    if pd.notna(df_pnr_pass["SSR_CODE_CD1"][j]) or pd.notna(df_pnr_pass["SPECIAL_NAME_CD2"][j] or pd.notna(df_pnr_pass["SPECIAL_NAME_CD1"][j])):
                        affect_person.iloc[i]["count_ssr"] += 1
        return affect_person

    def score_eval(affect_PNR):
        score = 0
        score += 200 * affect_PNR.loc["count_ssr"] + 50 * affect_PNR.loc["PAX_CNT"]
        score += 100 * (affect_PNR.loc["MAX_SEG"] - affect_PNR.loc["SEG_SEQ"])
        score_dict = {'A': 'J', 'D': 'J', 'J': 'J', 'B': 'F', 'F': 'F', 'S': 'Y', 'V': 'Y', 'W': 'Y', 'Z': 'Y', 'O': 'Y',
                      'S': 'Y', 'T': 'Y', 'U': 'Y', 'M': 'Y', 'N': 'Y', 'Y': 'Y'}
        if affect_PNR.loc["COS_CD"] not in score_dict.keys():
            score += 500
        elif affect_PNR.loc["COS_CD"] == 'J' or score_dict[affect_PNR.loc["COS_CD"]] == 'J':
            score += 2000
        elif affect_PNR.loc["COS_CD"] == "F" or score_dict[affect_PNR.loc["COS_CD"]] == "F":
            score += 1700
        elif affect_PNR.loc["COS_CD"] == "Y" or score_dict[affect_PNR.loc["COS_CD"]] == "Y":
            score += 1500
        elif affect_PNR.loc["COS_CD"] in ["A", "C", "K"]:
            score += 800
        return score

    def score(flight, data_time):
        affect_PNR = calc_ssr(flight, data_time)
        pnr_ranking = pd.DataFrame(columns=affect_PNR.columns.tolist() + ['score'])

        for i in range(len(affect_PNR)):
            score_val = score_eval(affect_PNR.iloc[i])
            affect_row = list(affect_PNR.iloc[i])
            affect_row.append(score_val)
            pnr_ranking = pnr_ranking.append(pd.Series(affect_row, index=pnr_ranking.columns), ignore_index=True)

        sorted_pnr_ranking = pnr_ranking.sort_values(by='score', ascending=False)
        return sorted_pnr_ranking

    return score(flight_number, date_time)

def get_cluster_rank(affect_sorted_PNR):
    cluster_means = affect_sorted_PNR.groupby('DEST_CD_ITN_DEST')['score'].mean()
    rank_mapping = {cluster: rank for rank, (cluster, _) in enumerate(cluster_means.sort_values(ascending=False).iteritems(), start=1)}
    affect_sorted_PNR['CLUSTER_RANK'] = affect_sorted_PNR['DEST_CD_ITN_DEST'].map(rank_mapping)
    return affect_sorted_PNR

def get_pnr_affected(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date):
    df_pnr_book = get_ITN_DEST(df_pnr_book)
    affect_sorted_PNR = process_pnr_data(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date)
    final_affected_PNR = get_cluster_rank(affect_sorted_PNR)

    return final_affected_PNR

## **INPUT BLOCK**


In [ ]:
flight_number = int(input("Enter the flight number: "))
tail_number = input("Enter the tail number: ")
date = input("Enter the date (MM/DD/YYYY): ")
affect_sorted_PNR = get_pnr_affected(df_pnr_pass, df_pnr_book, df_schedule, flight_number, tail_number, date)

**Storing Affected PNR Record**

In [6]:
affect_sorted_PNR.to_csv('/content/affected_PNR.csv', index=False)

## Generating Feasiable Flights from Source to conncetion and connection to destination

Using the Branch Prunning Algorithm and DAG (Direct Acyclic Graph) to schedule a connecting flight between the source and the destination in order to avoid taking the duplicated and impractical route


In [77]:
df=pd.read_csv("/content/drive/MyDrive/TEAM-69/Final Dataset/SCH-ZZ-20231208_035117.csv")
dff=pd.read_csv("/content/drive/MyDrive/TEAM-69/Final Dataset/INV-ZZ-20231208_041852.csv")

def source_flights(flt_num,tale_num,date):
  for i in range(len(df)):
    if(flt_num==df.loc[i]["FlightNumber"] and tale_num==df.loc[i]["AircraftTailNumber"]):
      return df.iloc[i]
  return -1

def conv_dep_time(a):
  k=0
  li=[]
  t=""
  for i in range(len(a)):
    if(a[i]=="'"):
      k=(k+1)%2
      if(k==0):
        li.append(t)
        t=""
    else:
      if(k==1):
        t+=a[i]
  return li

class DirectedWeightedGraph:
    def __init__(self):
        self.graph = {}

    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = []

    def add_edge(self, src, dest, weight):
        if src in self.graph:
            self.graph[src].append([dest, weight])
        else:
            self.graph[src] = [[dest, weight]]

    def get_adjacency_elements(self, vertex):
        adjacency_elements = set()
        if vertex in self.graph:
            adjacency_list = self.graph[vertex]
            for neighbor, _ in adjacency_list:
                adjacency_elements.add(neighbor)
        else:
            print(f'Vertex {vertex} not found in the graph.')

        return list(adjacency_elements)

    def edges_to_dataframe(self):
        merged_row_df = pd.DataFrame()
        for vertex in self.graph:
            for edge in self.graph[vertex]:
                dest, weight = edge
                merged_df = pd.concat([weight[0], weight[1]], axis=0)# Remove duplicate columns
                merged_df = pd.DataFrame(merged_df).T
                merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
                merged_row_df = merged_row_df.append(merged_df, ignore_index=True)
        return merged_row_df


    def display(self):
        for vertex in self.graph:
            for edge in self.graph[vertex]:
                print(vertex, '->',''.join(map(str, edge[0])))
            # print(vertex, '->', ' -> '.join(map(str, self.graph[vertex])))

    def delete_vertices_without_elements(self, element1, element2):
        vertices_to_delete = []
        for vertex in self.graph:
            if(vertex!=element1 and vertex!=element2):
                adjacency = [edge[0] for edge in self.graph[vertex]]
                print(vertex)
                if(element2 not in adjacency):
                    vertices_to_delete.append(vertex)
                if(vertex not in self.get_adjacency_elements(element1)):
                    vertices_to_delete.append(vertex)
        for vertex in vertices_to_delete:
            del self.graph[vertex]
        for vertex in self.graph:
            self.graph[vertex] = [(dest, weight) for dest, weight in self.graph[vertex] if dest not in vertices_to_delete]

from datetime import datetime, timedelta
def is_within_n_days(departure_datetime, target_datetime, n):
    datetime_format = "%m/%d/%Y %H:%M"

    departure_datetime = datetime.strptime(departure_datetime, datetime_format)
    target_datetime = datetime.strptime(target_datetime, datetime_format)

    days_difference = (target_datetime - departure_datetime).days

    return 0 <= days_difference <= n

def Graph_Flight_Formulation(df,dff,Soure,Destination,DD,n):
    for i in range(len(df)):
        print(i)
        if(df.loc[i]["DepartureAirport"].replace(" ","")==Source.replace(" ","") and df.loc[i]["ArrivalAirport"].replace(" ","")!=Destination.replace(" ","")):
            for j in range(len(dff)):
                for time in conv_dep_time(df.loc[i]["DepartureDates"]):
                    if(dff.loc[j]["ScheduleId"]==df.loc[i]["ScheduleID"] and dff.loc[j]["DepartureDate"]==time):
                        # print(df.loc[i]["DepartureAirport"],df.loc[i]["ArrivalAirport"])
                        g.add_edge(df.loc[i]["DepartureAirport"].replace(" ",""),df.loc[i]["ArrivalAirport"].replace(" ",""),[df.loc[i],dff.loc[j]])
                        g.add_vertex(df.loc[i]["ArrivalAirport"].replace(" ",""))
        if(df.loc[i]["DepartureAirport"].replace(" ","")!=Source.replace(" ","") and df.loc[i]["ArrivalAirport"].replace(" ","")==Destination.replace(" ","")):
            for k in range(len(dff)):
                for time1 in conv_dep_time(df.loc[i]["DepartureDates"]):
                    # print(dff.loc[k]["DepartureDate"],time1)
                    if(dff.loc[k]["ScheduleId"]==df.loc[i]["ScheduleID"] and dff.loc[k]["DepartureDate"]==time1):
                        # print(df.loc[i]["DepartureAirport"],df.loc[i]["ArrivalAirport"])
                        g.add_edge(df.loc[i]["DepartureAirport"].replace(" ",""),df.loc[i]["ArrivalAirport"].replace(" ",""),[df.loc[i],dff.loc[k]])
                        g.add_vertex(df.loc[i]["ArrivalAirport"].replace(" ",""))
    g.delete_vertices_without_elements(Source.replace(" ",""),Destination.replace(" ",""))
    res=(g.edges_to_dataframe())
    srcc=pd.DataFrame(columns=res.columns)
    destc=pd.DataFrame(columns=res.columns)
    for i in range(len(res)):
        if(res.loc[i]["DepartureAirport"].replace(" ","")==Source.replace(" ","")):
            srcc=srcc.append(res.loc[i])
        else:
            destc=destc.append(res.loc[i])
    for i in range(len(srcc)):
        t=""
        t+=srcc.loc[i]["DepartureDate"]
        t+=" "
        t+=srcc.loc[i]["DepartureTime"]
        if not is_within_n_days(DD,t,n):
            srcc=srcc.drop(i)
    return srcc,destc

**Source and Destination of Disturbed Iternary**

In [ ]:
ans=source_flights(flight_number,tail_number,date)
Source=ans["DepartureAirport"]
Destination=ans["ArrivalAirport"]
DD=date+" "+ans["DepartureTime"]
print("Disturbed Iternary",Source,"-->",Destination)

Getting Possible flights from source to connecting within n days of departure that is matching_s2c_flights

All possible flight from connecting to destination that is mamatching_c2d_flights

In [ ]:
g=DirectedWeightedGraph()
n=int(input("Minimum number of days for accomodation="))
matching_s2c_flights,matching_s2d_flights=Graph_Flight_Formulation(df,dff,Source,Destination,DD,n)
matching_s2c_flights=matching_s2c_flights.sort_values(by='DepartureDateTime')

# Scheduling of PNR into connecting Flights

**Affected PNR distribution into two flights based on class and arrival/departure time differences One involves connecting from the source to the destination, while the other is the opposite.**

In [114]:
# Assuming 'affect_sorted_PNR' is your DataFrame
df_FC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'FirstClass'].copy()
df_BC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'BusinessClass'].copy()
df_PC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'PremiumEconomyClass'].copy()
df_EC = affect_sorted_PNR[affect_sorted_PNR['COS_CD'] == 'EconomyClass'].copy()
print("passenger total in different class = ",df_FC.shape, df_BC.shape, df_PC.shape, df_EC.shape)
def knapsack1(left_seats, n, li, PNR_df, memo):
    if n == 0 or left_seats == 0:
        return left_seats, li

    # Check if the result is already memoized
    if (left_seats, n) in memo:
        return memo[(left_seats, n)]

    if left_seats >= PNR_df.iloc[n-1]["PAX_CNT"]:
        # Recursively compute the result and memoize it
        result1 = knapsack1(left_seats, n-1, li, PNR_df, memo)
        result2 = knapsack1(left_seats - PNR_df.iloc[n-1]["PAX_CNT"], n-1, li + [PNR_df.iloc[n-1]["RECLOC"]], PNR_df, memo)
        result = min(result1, result2, key=lambda x: x[0])
    else:
        result = knapsack1(left_seats, n-1, li, PNR_df, memo)

    # Memoize the result
    memo[(left_seats, n)] = result

    return result

def delete_entry(to_delete, dataframe):
    for i in to_delete:
        dataframe.drop(dataframe[dataframe["RECLOC"]==i].index, inplace=True, axis=0)
    return dataframe

def all_flights_knapsack_for_same_class1(df_PNR, flights, cls):
    allocated_dict = {}
    for i in range(len(flights)):
        seats_left, allocated_PNRs = knapsack1(flights.iloc[i][cls+"_AvailableInventory"], len(df_PNR), [], df_PNR, {})

        # Update the values in the flights DataFrame
        temp_column = cls+"_AvailableInventory"
        temp = flights.iloc[i][temp_column]
        column_index = flights.columns.get_loc(temp_column)
        flights.iat[i, column_index] = seats_left
        flights.iat[i, flights.columns.get_loc("AvailableInventory")] -= temp - seats_left
        df_PNR = delete_entry(allocated_PNRs, df_PNR)
        allocated_dict[flights.iloc[i]["InventoryId"]] = [(pnr, cls,flights.iloc[i]["DepartureDateTime"],flights.iloc[i]["ArrivalDateTime"],flights.iloc[i]["ArrivalAirport"]) for pnr in allocated_PNRs]

    return allocated_dict

def merge_dict(dict1,dict2):
    list1=dict1.keys()

    for i in list1:
        dict1[i]=dict1[i]+dict2[i]
    return dict1

def convert_dict_to_dataframe(allocated_dict):
    data = []

    for inventory_id, allocations in allocated_dict.items():
        for allocation in allocations:
            pnr, class_name , DT , AT, AA = allocation
            data.append({'PNR': pnr, 'InventoryID': inventory_id, 'New_ClassName': class_name, 'DT':DT,'AT':AT,"AA":AA})

    df = pd.DataFrame(data)
    return df

def complete_allocation(df_FC_copy,df_BC_copy,df_PE_copy,df_EC_copy,flights):
    print(flights)
    FC_allocation=all_flights_knapsack_for_same_class1(df_FC_copy, flights, "FC")
    BC_allocation=all_flights_knapsack_for_same_class1(df_BC_copy, flights, "BC")
    PE_allocation=all_flights_knapsack_for_same_class1(df_PE_copy, flights, "PC")
    EC_allocation=all_flights_knapsack_for_same_class1(df_EC_copy, flights, "EC")
    # print(PE_allocation)
    # print(EC_allocation)
    # print(BC_allocation)
    # print(FC_allocation)
    if not df_FC_copy.empty:
        PE_allocation.update(all_flights_knapsack_for_same_class1(df_FC_copy, flights, "PC"))
        EC_allocation.update(all_flights_knapsack_for_same_class1(df_FC_copy, flights, "EC"))
    if not df_BC_copy.COS_CD.empty:
        EC_allocation.update(all_flights_knapsack_for_same_class1(df_BC_copy, flights, "EC"))
    if not df_PE_copy.empty:
        FC_allocation.update(all_flights_knapsack_for_same_class1(df_PE_copy, flights, "FC"))
    if not df_EC_copy.empty:
        FC_allocation.update(all_flights_knapsack_for_same_class1(df_EC_copy, flights, "FC"))
        BC_allocation.update(all_flights_knapsack_for_same_class1(df_EC_copy, flights, "BC"))
        PE_allocation.update(all_flights_knapsack_for_same_class1(df_EC_copy, flights, "PC"))


    df=convert_dict_to_dataframe(FC_allocation)
    df=pd.concat([df, convert_dict_to_dataframe(BC_allocation)], ignore_index=True)
    df=pd.concat([df, convert_dict_to_dataframe(PE_allocation)], ignore_index=True)
    df=pd.concat([df, convert_dict_to_dataframe(EC_allocation)], ignore_index=True)
    print(df.shape,"#")
    return df

passenger total in different class =  (5, 22) (9, 22) (7, 22) (11, 22)


## **It return Schdule of affected PNR in dfn from source to connecting**

In [ ]:
dfn=complete_allocation(df_FC,df_BC,df_PC,df_EC,matching_s2c_flights)
dfn

**Merging the Schedule of PNR with thier details and clustering them on basis of the invetory ID of connecting flight from source to destination as a dataframe in a list named grouped_dfs**

In [116]:
final_df = pd.merge(dfn, affect_sorted_PNR, left_on='PNR', right_on='RECLOC', how='inner')
grouped_dfs = [group_df for _, group_df in final_df.groupby('InventoryID')]

### **Scheduling Flight of impacted PNR from connection to Destination**

In [132]:
def change_format(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    formatted_date = date_obj.strftime('%m/%d/%Y %H:%M')
    return formatted_date


In [133]:
def process_group(group_df, matching_s2d_flights):
    # Separate the group_df based on 'COS_CD'
    df_FC = group_df[group_df['COS_CD'] == 'FirstClass'].copy()
    df_BC = group_df[group_df['COS_CD'] == 'BusinessClass'].copy()
    df_PC = group_df[group_df['COS_CD'] == 'PremiumEconomyClass'].copy()
    df_EC = group_df[group_df['COS_CD'] == 'EconomyClass'].copy()

    # Call the allocation function for each segment
    c2d=matching_s2d_flights
    c2d.reset_index(drop=True, inplace=True)
    DD=change_format(group_df.iloc[0]["AT"])
    for i in range(len(c2d)):
        t=change_format(c2d.loc[i]["DepartureDateTime"])
        if not is_within_n_days(DD,t,3) or c2d.loc[i]["DepartureAirport"].replace(" ","")!=group_df.iloc[0]["AA"].replace(" ",""):
             c2d=c2d.drop(i)
    # print(c2d.shape,"@")
    c2d=c2d.sort_values(by='DepartureDateTime')
    # print(c2d.shape)
    df_result = complete_allocation(df_FC, df_BC, df_PC, df_EC, c2d)

    return df_result

In [134]:
combined_result_df = pd.DataFrame()

In [ ]:
for idx, group_df in enumerate(grouped_dfs):
    result_df = process_group(group_df, matching_s2d_flights)
    if(result_df.shape[0]==0):
        continue
    combined_result_df = pd.concat([result_df,combined_result_df], ignore_index=True)

**Number Allocated in Second Journey**

In [ ]:
print("Number of Passenger Allocated = ",combined_result_df.shape[0])
passengers_not_scheduled = affect_sorted_PNR[~affect_sorted_PNR['RECLOC'].isin(combined_result_df['PNR'])]
print("Passenger not get Scheduled in second Journey =",passengers_not_scheduled.shape[0])

Thus, we are taking into account that, even in the unlikely event that it is conceivable, passengers who are not scheduled for the connecting flight from the connecting airport to the destination will not receive any possible itinerary for the trip from the source to the connection.


**Storing 1st and 2nd Iternary of imapacted passenger**

In [138]:
passengers_scheduled_j1 = dfn[dfn['PNR'].isin(combined_result_df['PNR'])]
passengers_scheduled_j1.to_csv("/content/Journey1.csv",index=False)
combined_result_df.to_csv("/content/Journey2.csv")

# **END**